<a href="https://colab.research.google.com/github/asmundur/deeplearning-finalproject/blob/master/deep_learning_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from googleapiclient.discovery import build
import io, os, cv2, csv
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

from keras import regularizers
from keras.models import Sequential
from keras.layers.core import Dense, SpatialDropout1D
from keras.optimizers import SGD, rmsprop, Adam, Nadam
from sklearn.metrics import classification_report
from keras.layers import Conv2D, Flatten, MaxPooling2D, Activation, Dropout, AveragePooling2D
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [4]:
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)



!pip install kaggle


Download 100%.


In [0]:
# if not os.path.exists('./data'):
!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json
if (not os.path.exists('./train')) and (not os.path.exists('./test')):
  !kaggle competitions download -c humpback-whale-identification
  !unzip -q train.zip -d train
  !unzip -q test.zip -d test
  !ls train | wc -l
  !ls test | wc -l
  


In [0]:
# #!ls -a /root/.kaggle

# #!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json
# if not os.path.exists('./train'):
#   !unzip -q train.zip -d train

# !ls train | wc -l

# if not os.path.exists('./test'):
#   !unzip -q test.zip -d test

# !ls test | wc -l

# Data preparation

In [3]:
def getDictFromCsv(filename):
  reader = csv.reader(open(filename, 'r'))
  d = {}
  for row in reader:
    k, v = row
    d[k] = v
  return d

train_image_paths = list()
test_image_paths = list()
for (dirpath, dirnames, filenames) in os.walk("train"):
    for file in filenames:
        if '.jpg' in file and not file.startswith('.'):
              train_image_paths.append(os.path.join(dirpath, file))
test_image_paths = list()
for (dirpath, dirnames, filenames) in os.walk("test"):
    for file in filenames:
        if '.jpg' in file and not file.startswith('.'):
              test_image_paths.append(os.path.join(dirpath, file))

trainX  = []
testX   = []
trainY  = []
# testY   = []

input_labels = []
train_dict = getDictFromCsv('train.csv')
# test_dict = getDictFromCsv('test.csv')

# count = 0
for image_path in train_image_paths:
    image = cv2.imread(image_path)
    label = train_dict[image_path.split(os.path.sep)[-1]]
    trainX.extend(image)
    trainY += [label]*len(image)
#     print([label]*len(image))
#     print(len([label]*len(image)))
#     print(label)
#     original_imgs.append(image)
#     if count < test_breakoff:
#       images = preprocess_img(image, sz)
#       trainX.extend(images)
#       trainY += [label]*len(images)
#       print([label]*len(images))
#       print(len([label]*len(images)))
#     else:
#       image = cv2.resize(image, dsize=(sz, sz), interpolation=cv2.INTER_NEAREST)
#       testX.append(image)
#       testY.append(label)
#     count += 1

for image_path in test_image_paths:
    image = cv2.imread(image_path)
#     label = train_dict[image_path.split(os.path.sep)[-1]]
    testX.extend(image)

# Convert the target categorial labels into binary vectors 
# (for 2-class, binary classification you should use Keras' 
#  to_categorical function instead as the scikit-learn's LabelBinarizer)
lb = LabelBinarizer()    # ... from scikit
trainY = lb.fit_transform(trainY)  # ... from scikit
trainX = np.array(trainX)
testX = np.array(testX)

KeyboardInterrupt: ignored